In [31]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import sqlite3
import numpy as np


In [2]:
#Load the .csv'sas Panda's Data Frames
tops = pd.read_csv("top_airports.csv")
ICAO = pd.read_csv("ICAO_airports.csv")
top_50 = pd.read_csv("AirportData.csv")

In [ ]:
#convert to sqlite3 databases

#initialize the databases and connection files
conn1 = sqlite3.connect("/tmp/top_airports.db")
conn2 = sqlite3.connect("/tmp/ICAO_airports.db")
conn3 = sqlite3.connect("/tmp/AirportData.db")

cursor1 = conn1.cursor()
cursor2 = conn2.cursor()
cursor3 = conn3.cursor()

In [ ]:
#convert from Pandas data frame to SQL data base - Only need to run this cell once... ever
tops.to_sql('tops_db', conn1)
ICAO.to_sql('ICAO_db', conn2)
top_50.to_sql('AirportData_db', conn3)

In [ ]:
# #Lets check if the database was created
# sql_cmd = "SELECT * FROM AirportData_db"
# cursor3.execute(sql_cmd)

# db_info = cursor3.fetchall()
# for entry in db_info: 
#      print(entry[:8])

In [ ]:
#run to close the connection to the data base
conn3.commit()
conn3.close()

In [74]:
#Define the web crawler function
def weather_history(code):
   
    #ideally one would use the following lines to autofill dates, however, the site is incomplete 
    #for recent dates so the dates are set to a range that has been tested (last month)
    
    now = datetime.datetime.now() 
    month = str(now.month) 
    day = str(now.day-7) 

#assemble url
# use this form if auto selecting the past week works
# url = 'https://www.wunderground.com/history/airport/' + code + '/2018/' + month + '/' + 'day' + '/WeeklyHistory.html?&reqdb.zip=&reqdb.magic=&reqdb.wmo='
    url = 'https://www.wunderground.com/history/airport/' + code + '/2018/2/1/MonthlyHistory.html?&reqdb.zip=&reqdb.magic=&reqdb.wmo='
    
    #call and read with BS4
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')

    #parse the table
    table = soup('table')[3]
    table_body = table.find_all('tbody')
    table_head = table.find('thead')

    header = []

    for th in table_head.findAll('th'):
        key = th.get_text()
        header.append(key)

    # The first row has an odd sub-header, so it will be left out and re-genreated manually when creating the pandas df
    rows = []
    for tb in table_body[1:]:
        row = []
        for elem in tb.find_all('td'):
            try:
                row.append(float(elem.get_text().strip()))
            except ValueError:
                pass

        rows.append(row)
            
    #define the column headers
    column_head=[ 'February','High Temp','Avg Temp','Low Temp','Low Dew Point','Avg Dew Point','High Dew Point','High Humidity (%)','Avg Humidity (%)','Low HUmidity (%)',
            'High Sea Level Press. (in)','Avg Sea Level Press. (in)','Low Sea Level Press. (in)','High Visibiity (mi)','Avg Visibiity (mi)','Low Visibiity (mi)',
            'High Wind (mph)','Avg High Wind (mph)','Low High Wind (mph)','Precip. (in)']
    df=pd.DataFrame(rows,columns=column_head)
    
    return df

In [89]:
df = weather_history('KSFO')


1.0

In [90]:
df

,February,High Temp,Avg Temp,Low Temp,Low Dew Point,Avg Dew Point,High Dew Point,High Humidity (%),Avg Humidity (%),Low HUmidity (%),High Sea Level Press. (in),Avg Sea Level Press. (in),Low Sea Level Press. (in),High Visibiity (mi),Avg Visibiity (mi),Low Visibiity (mi),High Wind (mph),Avg High Wind (mph),Low High Wind (mph),Precip. (in)
0,1.0,73.0,62.0,51.0,51.0,46.0,39.0,89.0,59.0,29.0,30.14,30.10,30.06,10.0,10.0,10.0,13.0,4.0,15.0,0.00
1,2.0,73.0,62.0,51.0,53.0,47.0,43.0,89.0,63.0,37.0,30.19,30.14,30.11,10.0,10.0,10.0,15.0,3.0,16.0,0.00
2,3.0,76.0,63.0,50.0,49.0,47.0,44.0,83.0,59.0,35.0,30.17,30.12,30.08,10.0,10.0,10.0,16.0,5.0,20.0,0.00
3,4.0,75.0,63.0,51.0,56.0,49.0,45.0,80.0,58.0,35.0,30.14,30.08,30.03,10.0,10.0,10.0,16.0,4.0,22.0,0.00
4,5.0,72.0,63.0,53.0,55.0,50.0,47.0,89.0,69.0,49.0,30.06,30.02,29.96,10.0,10.0,10.0,23.0,6.0,27.0,0.00
5,6.0,75.0,64.0,52.0,48.0,39.0,28.0,77.0,48.0,19.0,30.14,30.06,30.01,10.0,10.0,10.0,8.0,2.0,12.0,0.00
6,7.0,72.0,62.0,51.0,51.0,46.0,36.0,72.0,55.0,38.0,30.26,30.21,30.14,10.0,10.0,9.0,12.0,2.0,13.0,0.00
7,8.0,74.0,63.0,51.0,53.0,48.0,40.0,89.0,60.0,31.0,30.24,30.17,30.09,10.0,9.0,4.0,14.0,4.0,17.0,0.00
8,9.0,74.0,63.0,51.0,52.0,47.0,40.0,77.0,58.0,38.0,30.10,30.03,29.97,10.0,10.0,10.0,23.0,8.0,27.0,0.00
9,10.0,65.0,59.0,53.0,51.0,44.0,28.0,83.0,55.0,27.0,30.00,29.96,29.91,10.0,10.0,8.0,24.0,14.0,29.0,0.00


In [103]:
#Generate the Second Empty Table

conn4 = sqlite3.connect("/tmp/weathers2.db") #will create the file if it doesnt alredy exist

cursor4 = conn4.cursor()

sql_cmd = """CREATE TABLE weathers2 (
id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    day INT,
    min_temp FLOAT,
    max_temp FLOAT,
    humitidy FLOAT,
    precipitation FLOAT)"""


cursor4.execute(sql_cmd) #just creates the table


In [105]:
#use web crawler function to populate the database - may take some time
for ii in range(len(top_50)):
    if top_50.iloc[ii,1] == 'HNL': #manually check bc wunderground doesnt recognize these IATA codes normally
        code = top_50.iloc[ii,1] 
    elif top_50.iloc[ii,1] =='SJU':
        code = top_50.iloc[ii,1]
    else: 
        code = 'K' + top_50.iloc[ii,1] #K's are added to the IATA code for wunderground to recognize
    if code == 'KSMF': #bc Sacremento's wunderground table HTML is different from the rest and its currently 1:30 am, skip it.. 
        continue
        
    df = weather_history(code) #rn this only runs 1 week back, not 10 years
    df = df.fillna('0') #replaces NaN's with back-filled data (NaN's where data is missing from website)
    #inserts the following data which is referenced from the structure of the data frame from the web crawler 
    for jj in range(len(df)):
        airport_data = (top_50.iloc[ii,0],df.iloc[jj,0],df.iloc[jj,3],df.iloc[jj,2],df.iloc[jj,8],df.iloc[jj,19])
        sql_cmd = ("INSERT INTO weathers2 (name, day, min_temp, max_temp, humitidy, precipitation) VALUES" + str(airport_data))
        cursor4.execute(sql_cmd)

In [108]:
# #check if entry worked - run simple query
# sql_cmd = "SELECT * FROM weathers2 WHERE name = 'Hartsfield_Jackson_Atlanta_International'"
# cursor4.execute(sql_cmd)

# db_info = cursor4.fetchall()
# for entry in db_info: 
#     print(entry)
    
# # connection.commit()
# # connection.close()

In [116]:
# Extract Data from sqlib as usable form for correlation calculations

mintemp_dict={} #pre-define dictionary to store results from all airports
maxtemp_dict={}
#humid_dict={}
precip_dict={}

# 1 day comparisons
for ii in range(len(top_50)):
    if top_50.iloc[ii,1] == 'SMF': #bc Sacremento's was not included above, skip
        continue
    code = str(top_50.iloc[ii,0])
    mintemp_cmd = ("SELECT weathers2.min_temp FROM weathers2 WHERE name = '" + str(code) + "'")
    maxtemp_cmd = ("SELECT weathers2.max_temp FROM weathers2 WHERE name = '" + str(code) + "'")
    precip_cmd = ("SELECT weathers2.precipitation FROM weathers2 WHERE name = '" + str(code) + "'")
    
    #extract min temp data
    cursor4.execute(mintemp_cmd)
    db_info = cursor4.fetchall()
    
    #convert the returned tuple to a useable array format
    db_str = str(db_info)
    end_db_str=db_str[2:-2].split(',')
    my_array = [float(i) for i in end_db_str]
    my_array = np.asarray(my_array)
    aprt = str(code)
    mintemp_dict[aprt] = my_array
    
    #extract min temp data
    cursor4.execute(maxtemp_cmd)
    db_info = cursor4.fetchall()
    
    #convert the returned tuple to a useable array format
    db_str = str(db_info)
    end_db_str=db_str[2:-2].split(',')
    my_array = [float(i) for i in end_db_str]
    my_array = np.asarray(my_array)
    aprt = str(code)
    maxtemp_dict[aprt] = my_array
    
    
    #extract preip. data
    cursor4.execute(precip_cmd)
    db_info = cursor4.fetchall()
    
    #convert the returned tuple to a useable array format
    db_str = str(db_info)
    end_db_str=db_str[2:-2].split(',')
    my_array = [float(i) for i in end_db_str]
    my_array = np.asarray(my_array)
    aprt = str(code)
    precip_dict[aprt] = my_array
    
    

ValueError: could not convert string to float: ')'

In [117]:
db_info

[(40.0,),
 (32.0,),
 (32.0,),
 (35.0,),
 (38.0,),
 (40.0,),
 (41.0,),
 (33.0,),
 (37.0,),
 (51.0,),
 (61.0,),
 (50.0,),
 (47.0,),
 (48.0,),
 (59.0,),
 (55.0,),
 (51.0,),
 (48.0,),
 (59.0,),
 (58.0,),
 (65.0,),
 (63.0,),
 (60.0,),
 (63.0,),
 (59.0,),
 (49.0,),
 (45.0,),
 (51.0,)]

In [119]:
mintemp_cmd = ("SELECT weathers2.min_temp FROM weathers2 WHERE name = 'Orlando_International'")
cursor4.execute(mintemp_cmd)
db_info = cursor4.fetchall()

In [131]:
db_str = str(db_info)
end_db_str=db_str[1:-1]
end_db_str

'(49.0,), (56.0,), (55.0,), (59.0,), (53.0,), (52.0,), (62.0,), (62.0,), (60.0,), (62.0,), (66.0,), (69.0,), (65.0,), (61.0,), (61.0,), (58.0,), (57.0,), (54.0,), (63.0,), (67.0,), (67.0,), (66.0,), (65.0,), (63.0,), (63.0,), (69.0,), (62.0,), (58.0,)'

In [78]:
#CLEANER - Only Call if ABS necessary

def delete_all_tasks(conn):
    """
    Delete all rows in the tasks table
    :param conn: Connection to the SQLite database
    :return:
    """
    sql = 'DELETE FROM weathers' #change 'weathers' to which ever database needs cleaning 'AirportData_db'
    cur = conn.cursor()
    cur.execute(sql)

delete_all_tasks(conn4)